<h1 style="font-size: 40px; font-weight: bold; color: blue; text-align: center; line-height: 1.2;">
  Représentation des importations et des exportations du Canada (1971 - 2012)
</h1>

lien DataSet : https://www150.statcan.gc.ca/n1/tbl/csv/12100014-fra.zip

# Étapes


##  Extraction
1. **Télécharger le fichier ZIP**  
2. **Extraction et affichage des CSV Bruts** 
3. **Identifier le bon tableau contenant les bonnes colonnes**
4. **Vérification du tableau trouvé**
5. **Extraction des données**
---

##  Transformation

1. **Selection du CSV '12100014.csv'**
2. **Vérification de l'existence du fichier cible dans l'archive ZIP**
3. **Nettoyage des noms de colonnes**
4. **Selection des colonnes utilies**
5. **Filtrage des colonnes**
6. **Nettoyage des chaînes de caractères**
7. **Remplacement des valeurs numériques manquantes par la médiane**
8. **Supprimer les doublons**
9. **Formatage des dates**
10. **Réinitialisation de l'index**
11. **Enregistrement du DataFrame nettoyé dans le dictionnaire**
---

##  Chargement (Laoding)
1. **Connexion SQL Server**
2. **Insertion dans SQL Server**

---


---
##  Extraction

In [2]:
import sys
import requests
import zipfile
import io
import pandas as pd
from IPython.display import display
from sqlalchemy import create_engine
import urllib

# Étape 1 : Télécharger le fichier ZIP
url = "https://www150.statcan.gc.ca/n1/tbl/csv/12100014-fra.zip"
response = requests.get(url)

if response.status_code != 200:
    print(f"Erreur de téléchargement. Code d'erreur : {response.status_code}")
    sys.exit()

# Étape 2 : Extraction et affichage des CSV Bruts
with zipfile.ZipFile(io.BytesIO(response.content)) as archive_zip:
    csv_files = [f for f in archive_zip.namelist() if f.endswith('.csv')]

    # Affichage du CSV Brut
    print("Fichiers CSV dans le ZIP :")
    for csv_name in csv_files:
        with archive_zip.open(csv_name) as file:
            raw_content = file.read().decode('utf-8')
            print(f"\n --- APERCU DU FICHIER CSV BRUT : {csv_name} ---")
            print("\n".join(raw_content.splitlines()[:10]))  # 10 premières lignes


Fichiers CSV dans le ZIP :

 --- APERCU DU FICHIER CSV BRUT : 12100014.csv ---
﻿"PÉRIODE DE RÉFÉRENCE";"GÉO";"DGUID";"Base";"Désaisonnalisation";"L'importation et l'exportation, groupes principaux de marchandises et marchés principaux";"UNITÉ DE MESURE";"IDENTIFICATEUR D'UNITÉ DE MESURE";"FACTEUR SCALAIRE";"IDENTIFICATEUR SCALAIRE";"VECTEUR";"COORDONNÉES";"VALEUR";"STATUS";"SYMBOLE";"TERMINÉ";"DÉCIMALES"
"1971-01";"Canada";"2016A000011124";"Balance des paiements";"Non désaisonnalisées";"Importations, total de toutes les marchandises";"Dollars";"81";"millions";"6";"v183258";"1.2.1.1";"3349.0";"";"";"";"1"
"1971-01";"Canada";"2016A000011124";"Balance des paiements";"Non désaisonnalisées";"Importations, section 1 Animaux vivants";"Dollars";"81";"millions";"6";"v183259";"1.2.1.2";"18.5";"";"";"";"1"
"1971-01";"Canada";"2016A000011124";"Balance des paiements";"Non désaisonnalisées";"Importations, section 2 Aliments, provendes, boissons et tabacs";"Dollars";"81";"millions";"6";"v183260";"1.2

---
##  Transformation

In [6]:

# Selection du CSV '12100014.csv'
cleaned_dfs = {}

with zipfile.ZipFile(io.BytesIO(response.content)) as archive_zip:
    target_csv = "12100014.csv"

    # Vérification de l'existence du fichier cible dans l'archive ZIP
    if target_csv in archive_zip.namelist():


        with archive_zip.open(target_csv) as file:
            df = pd.read_csv(file, sep=';', encoding='utf-8')

        # Nettoyage des noms de colonnes
        df.columns = (
            df.columns
            .str.strip()
            .str.lower()
            .str.replace(' ', '_')
            .str.replace("'", '')
            .str.replace(',', '')
        )

        # Selection des colonnes à conserver
        colonnes_conservees = [
            'période_de_référence',
            'valeur',
            'limportation_et_lexportation_groupes_principaux_de_marchandises_et_marchés_principaux',
            'unité_de',
            'base'
        ]

        # Filtrage des colonnes
        colonnes_presentes = [col for col in colonnes_conservees if col in df.columns]
        df = df[colonnes_presentes]

        # Nettoyage des chaînes de caractères
        for col in df.select_dtypes(include='object').columns:
            df[col] = df[col].str.strip().replace('', None)

        # Remplacement des valeurs numériques manquantes par la médiane
        for col in df.select_dtypes(include='number').columns:
            if df[col].isnull().any():
                df[col] = df[col].fillna(df[col].median())
        
        # Supprimer les doublons
        df = df.drop_duplicates()

        # Formatage des dates
        if 'période_de_référence' in df.columns:
            df['période_de_référence'] = pd.to_datetime(df['période_de_référence'], errors='coerce')

        # Réinitialisation de l'index
        df.reset_index(drop=True, inplace=True)

        # Enregistrement du DataFrame nettoyé dans le dictionnaire
        cleaned_dfs[target_csv] = df

        print(f"\n=== Données NETTOYÉES : {target_csv} ===")
        print(f"Colonnes conservées : {colonnes_presentes}")
        print(f"Nombre de lignes : {df.shape[0]}")
        print(f"Nombre de colonnes : {df.shape[1]}")
        display(df.info())

    else:
        print(f"Le fichier {target_csv} n'a pas été trouvé dans l'archive ZIP.")
        sys.exit()


=== Données NETTOYÉES : 12100014.csv ===
Colonnes conservées : ['période_de_référence', 'valeur', 'limportation_et_lexportation_groupes_principaux_de_marchandises_et_marchés_principaux', 'base']
Nombre de lignes : 77342
Nombre de colonnes : 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77342 entries, 0 to 77341
Data columns (total 4 columns):
 #   Column                                                                                 Non-Null Count  Dtype         
---  ------                                                                                 --------------  -----         
 0   période_de_référence                                                                   77342 non-null  datetime64[ns]
 1   valeur                                                                                 77342 non-null  float64       
 2   limportation_et_lexportation_groupes_principaux_de_marchandises_et_marchés_principaux  77342 non-null  object        
 3   base                        

None

---
##  Chargement (Laoding)

In [ ]:

# Connexion SQL Server
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=LAPTOP-7GECG3FQ\\SQLEXPRESS;"
    "DATABASE=DataBaseGouvernement;"
    "Trusted_Connection=yes;"
)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# Insertion dans SQL Server
for csv_name, df in cleaned_dfs.items():
    table_name = csv_name.replace('.csv', '').replace('-', '_').lower()
    print(f"\n Insertion des données dans la table SQL : {table_name}")
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f" Insertion terminée pour {table_name}")



 Insertion des données dans la table SQL : 12100014
 Insertion terminée pour 12100014
